In [1]:
# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# https://www.census.gov/data/datasets/2019/econ/cbp/2019-cbp.html

df_raw = pd.read_csv('./raw_data/county_business_profile_2019_raw.csv', na_values=".")


In [3]:
# Cleanup FIPS codes
df_raw['fips'] = df_raw['fipstate'] * 1000 + df_raw['fipscty']

In [4]:
# Renaming dataframe columns
# # https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/2018_record_layouts/county-layout-2018.txt

df = pd.DataFrame()
df['naics'] = df_raw['naics']

df['fips'] = df_raw['fips']
df['cbp_total_employees'] = df_raw['emp']
df['cbp_total_establishments'] = df_raw['est']

df['cbp_num_employees_5'] = df_raw['n<5']
df['cbp_num_employees_5_9']= df_raw['n5_9']
df['cbp_num_employees_10_19'] = df_raw['n10_19']
df['cbp_num_employees_20_49'] = df_raw['n20_49']
df['cbp_num_employees_50_99'] = df_raw['n50_99']
df['cbp_num_employees_100_249'] = df_raw['n100_249']
df['cbp_num_employees_250_499'] = df_raw['n250_499']
df['cbp_num_employees_500_999'] = df_raw['n500_999']
df['cbp_num_employees_1000'] = df_raw['n1000']


# df['cbp_num_employees_'] = df['N1000_1']
# df['cbp_num_employees_'] = df['N1000_2']
# df['cbp_num_employees_'] = df['N1000_3']
# df['cbp_num_employees_'] = df['N1000_4']


# N<5             N       Number of Establishments: Less than 5 Employee Size Class
# N5_9            N       Number of Establishments: 5-9 Employee Size Class
# N10_19          N       Number of Establishments: 10-19 Employee Size Class
# N20_49          N       Number of Establishments: 20-49 Employee Size Class
# N50_99          N       Number of Establishments: 50-99 Employee Size Class
# N100_249        N       Number of Establishments: 100-249 Employee Size Class
# N250_499        N       Number of Establishments: 250-499 Employee Size Class
# N500_999        N       Number of Establishments: 500-999 Employee Size Class
# N1000           N       Number of Establishments: 1,000 or More Employee Size Class
# N1000_1         N       Number of Establishments: Employment Size Class:
#                                 1,000-1,499 Employees
# N1000_2         N       Number of Establishments: Employment Size Class:
#                                 1,500-2,499 Employees
# N1000_3         N       Number of Establishments: Employment Size Class:
#                                 2,500-4,999 Employees
# N1000_4         N       Number of Establishments: Employment Size Class:
#                                 5,000 or More Employees

In [5]:
df

,naics,fips,cbp_total_employees,cbp_total_establishments,cbp_num_employees_5,cbp_num_employees_5_9,cbp_num_employees_10_19,cbp_num_employees_20_49,cbp_num_employees_50_99,cbp_num_employees_100_249,cbp_num_employees_250_499,cbp_num_employees_500_999,cbp_num_employees_1000
0,------,1001,11510,867,412,200,112,92,38,10,N,N,N
1,11----,1001,86,10,7,N,N,N,N,N,N,N,N
2,113///,1001,73,7,5,N,N,N,N,N,N,N,N
3,1133//,1001,73,7,5,N,N,N,N,N,N,N,N
4,11331/,1001,73,7,5,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085467,61131/,56999,18,3,N,N,N,N,N,N,N,N,N
1085468,611310,56999,18,3,N,N,N,N,N,N,N,N,N
1085469,62----,56999,15,3,N,3,N,N,N,N,N,N,N
1085470,621///,56999,15,3,N,3,N,N,N,N,N,N,N


In [6]:
# Types of businesses
# https://www2.census.gov/programs-surveys/cbp/technical-documentation/reference/naics-descriptions/naics2017.txt
df = df.query('naics == "------"')

In [7]:
df

,naics,fips,cbp_total_employees,cbp_total_establishments,cbp_num_employees_5,cbp_num_employees_5_9,cbp_num_employees_10_19,cbp_num_employees_20_49,cbp_num_employees_50_99,cbp_num_employees_100_249,cbp_num_employees_250_499,cbp_num_employees_500_999,cbp_num_employees_1000
0,------,1001,11510,867,412,200,112,92,38,10,N,N,N
320,------,1003,67084,5644,2989,1135,772,524,141,66,11,5,N
1176,------,1005,6893,449,255,99,55,22,9,5,N,N,N
1365,------,1007,3951,285,152,64,31,27,5,3,N,N,N
1517,------,1009,6825,695,380,152,88,52,20,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084310,------,56039,18721,2221,1519,306,211,135,32,14,N,N,N
1084832,------,56041,7513,566,314,121,67,45,11,5,N,N,N
1085091,------,56043,2483,338,208,63,43,17,5,N,N,N,N
1085274,------,56045,1737,216,130,42,24,17,N,3,N,N,N


In [8]:
# df[df.columns] = df[df.columns].replace(['N', 0], np.nan)
df[df.columns] = df[df.columns].replace(['N'], 0)


C:\Users\cjong\AppData\Local\Temp\ipykernel_15380\1024917313.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.columns] = df[df.columns].replace(['N'], 0)


In [9]:
df

,naics,fips,cbp_total_employees,cbp_total_establishments,cbp_num_employees_5,cbp_num_employees_5_9,cbp_num_employees_10_19,cbp_num_employees_20_49,cbp_num_employees_50_99,cbp_num_employees_100_249,cbp_num_employees_250_499,cbp_num_employees_500_999,cbp_num_employees_1000
0,------,1001,11510,867,412,200,112,92,38,10,0,0,0
320,------,1003,67084,5644,2989,1135,772,524,141,66,11,5,0
1176,------,1005,6893,449,255,99,55,22,9,5,0,0,0
1365,------,1007,3951,285,152,64,31,27,5,3,0,0,0
1517,------,1009,6825,695,380,152,88,52,20,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084310,------,56039,18721,2221,1519,306,211,135,32,14,0,0,0
1084832,------,56041,7513,566,314,121,67,45,11,5,0,0,0
1085091,------,56043,2483,338,208,63,43,17,5,0,0,0,0
1085274,------,56045,1737,216,130,42,24,17,0,3,0,0,0


In [10]:
# df = df[['fips', 'cbp_total_employees', 'cbp_total_establishments']]

In [11]:
df.to_csv('./processed_data/county_business_profile_2019.csv')